In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle

# Config Path 설정

In [2]:
import yaml

with open('config.yaml') as f:
    conf = yaml.full_load(f)
    

### [option]dataset 확인 (형태확인 및 피쳐수 확인 필요시)

In [3]:
with open(conf['dataset']['chat_path'],"rb") as f1:  
    chat=pickle.load(f1)

with open(conf['dataset']['audio_path'],"rb") as f2:  
    audio=json.load(f2)
with open(conf['dataset']['video_path'],"rb") as f2:  
    video=json.load(f2)
with open(conf['dataset']['label_path'],"rb") as f4:  
    real_result=pickle.load(f4)

In [4]:
len(audio['102844412722519367'])

2069

# Dataset

video 형태가 안맞았는지, 혼자만 코드 이상하게 되어있음. 

기본 형태 : 'gameid':[[0초 피쳐],[1초 피쳐]]

video는 1개의 피쳐가 그냥 'gameid':[0초피쳐,1초피쳐,,,,]로 되어있음.
 --> 추후 수정시, 

**s_window+=list(self.video[game_id][vframe+idx])**

로만 바꿔주시오~


In [5]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type

        with open(conf['dataset']['chat_path'],"rb") as f1:  
            self.chat=pickle.load(f1)
       
        with open(conf['dataset']['audio_path'],"rb") as f2:  
            self.audio=json.load(f2)
        with open(conf['dataset']['video_path'],"rb") as f2:  
            self.video=json.load(f2)
        with open(conf['dataset']['label_path'],"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.audio[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            window_size=conf['trn_args']['window_size']
            std_size = min(len(self.audio[game_id]),min(len(self.chat[game_id]),len(self.video[game_id]))) #데이터가 가장 짧은거에 맞춤 (오류방지)
            chat_size = conf['dataset']['chat_size']#c초당 피쳐 개수
            audio_size = conf['dataset']['audio_size']
            video_size = conf['dataset']['video_size']
            global total_size
            total_size = int(chat_size)+int(audio_size)+int(video_size)
            
            
            for idx in range(window_size): #7 : window size
                s_window=[]
                if vframe+idx<std_size:#아래는 데이터 형태가 조금씩 달라서 다음과 같이 진행.
                    s_window+=list((self.chat[game_id][vframe+idx]))#vframe의 chat
                    s_window+=list(self.audio[game_id][vframe+idx])#vframe의 image
                    s_window+=list(self.video[game_id][vframe+idx])
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*(total_size)#초가 초과되는 경우 0으로 피쳐 패
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [6]:
train=Mul_data('train')
val=Mul_data('val')

#test
print("\n\ndatset testing....")
print(train[0])
print(len(train[100][1]))
print("\n****************\n")


#loader 
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=conf['trn_args']['sampling'])
train_loader=torch.utils.data.DataLoader(train,batch_size=conf['trn_args']['trn_bs'],sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=conf['trn_args']['val_bs'])

data load fin
data load fin


datset testing....
('102844412722519367', array([[ 0.02620566,  0.0118512 , -0.02706682, ..., -0.02692822,
        -0.03071774, -0.02038495],
       [-0.06698951, -0.03389126, -0.00954571, ..., -0.02692822,
        -0.03071774, -0.02038495],
       [-0.06153676, -0.02784403, -0.01758359, ..., -0.02692822,
        -0.03071774, -0.02038495],
       ...,
       [-0.08774932, -0.03491368, -0.01236366, ..., -0.02692822,
        -0.03071774, -0.02038495],
       [-0.0150439 , -0.00701944, -0.01754227, ..., -0.02692822,
        -0.03071774, -0.02038495],
       [ 0.01076563,  0.00668381, -0.01694852, ..., -0.02692822,
        -0.03071774, -0.02038495]]), 0)
23

****************



# Modeling

In [7]:
global input_size,hidden_size,num_layers
input_size=total_size
hidden_size=conf['trn_args']['hidden_size']
num_layers=conf['trn_args']['num_layer']

print(input_size)
class LSTM(nn.Module):
    def __init__(self):
        
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,num_layers,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        feature = out[:,-1,:]
        out = self._lin(out[:,-1,:])
        return out,feature
        

384


In [8]:
lr = conf['trn_args']['lr']

model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr,momentum=0.9,weight_decay=1e-4)

# Experiment 

### measures

In [9]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [10]:
##best 값을 확인하기위함

def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

### experiment

In [11]:
global weight_dir
weight_dir= conf['dataset']['weight_path']##weight 저장
model_epoch = conf['trn_args']['epoch']

# dataset=weight_dir+'best'
# checkpoint=torch.load(dataset,map_location='cuda:0')
# model.load_state_dict(checkpoint)

if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(model_epoch):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out,_=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    
    
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out,_=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            

0
epoch 0 train_loss : 0 , val_loss : 0.6905721426010132,p 0, r 0, f 0

1
epoch 1 train_loss : 0 , val_loss : 0.7004979848861694,p 0, r 0, f 0

2
epoch 2 train_loss : 0 , val_loss : 0.6888850927352905,p 0, r 0, f 0

3
epoch 3 train_loss : 0 , val_loss : 0.6876826882362366,p 0, r 0, f 0

4
epoch 4 train_loss : 0 , val_loss : 0.6938268542289734,p 0.19273755554558103, r 0.9479028697571744, f 0.32034018426647765

5
epoch 5 train_loss : 0 , val_loss : 0.687271237373352,p 0, r 0, f 0

6
epoch 6 train_loss : 0 , val_loss : 0.6877807974815369,p 0.7, r 0.04481236203090508, f 0.08423236514522821

7
epoch 7 train_loss : 0 , val_loss : 0.6905197501182556,p 0.41522359657469077, r 0.4816777041942605, f 0.4459887583035258

8
epoch 8 train_loss : 0 , val_loss : 0.6992735266685486,p 0, r 0, f 0

9
epoch 9 train_loss : 0 , val_loss : 0.6965732574462891,p 0.17961634561134843, r 0.9838852097130243, f 0.3037758996728462

10
epoch 10 train_loss : 0 , val_loss : 0.6987226009368896,p 0.1749078921853791, r 0.9

#

### Evaluation

In [12]:
def fmeasure2(frames,label): ##measure def for test
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))


test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=conf['trn_args']['val_bs'])
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}


#evaluation
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output,_=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
with open(conf['dataset']['label_path'],"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

data load fin
102844212431058132
precision : 0.8393782383419689, recall : 0.4251968503937008, f1 : 0.5644599303135889, accuracy : 0.898538961038961
102844341902586509
precision : 0.5669099756690997, recall : 0.9209486166007905, f1 : 0.7018072289156627, accuracy : 0.9073900841908326
102844401152267937
precision : 0.3132780082987552, recall : 0.3756218905472637, f1 : 0.3416289592760181, accuracy : 0.745740498034076
102844212430927059
precision : 0.7455089820359282, recall : 0.7545454545454545, f1 : 0.7500000000000001, accuracy : 0.9127004996055745
102844412708953395
precision : 0.8027522935779816, recall : 0.6340579710144928, f1 : 0.708502024291498, accuracy : 0.9299610894941635
102844212429944013
precision : 0.7, recall : 0.5094339622641509, f1 : 0.5897035881435257, accuracy : 0.8714565004887586
102844341912679064
precision : 0.39552238805970147, recall : 0.9380530973451328, f1 : 0.5564304461942257, accuracy : 0.8257731958762886
102844235753749959
precision : 0.3148614609571788, recall 